**Objective:**

1. We use 'Coronal' plane to be able to get insight into sensitivity analysis.
2. Sensitivity analysis using 'val_accuracy',  'val_recall' and  'val_precision' on the y-axis
3. Choices of x-axis are:  
    a) Filter kernel sizes in 1st layer - 3,5,7,9,11  
    b) Number of filter sizes in 1st layer - 8,16,32,64,128,256 (show in log scale)  
    c) Neurons in fully connected layer - 16,32,64,128,256  
    d) Learning rate of model -  0.0001,0.001,0.01,0.1,1
    e) Regularizer values applied in each layer - 
4. We ensure to use 5-fold runs for each one.
4. Final results are pickled into 'sen_analysis' folder in shared space after reach run.

In [ ]:
# Env variable to optimize memory usage
import os
import datetime
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models,preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import f1_score, confusion_matrix,precision_score,recall_score,accuracy_score,log_loss
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from IPython.display import display_html 

In [ ]:
# Define base path where files will be stored.
# This is unpacked from the pickle file created in Step 0.

with open('pickledHomeScratchShared.pickle', "rb") as f:
    baseHomePath,baseScratchPath,baseSharedPath = pickle.load(f)

In [ ]:
# Set memory growth on GPUs (another step for memory optimization)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
  except RuntimeError as e:
    print("Error: "+e)

#Print GPU devices, if any, that are available
physical_devices = tf.config.list_physical_devices('GPU')
print("Physical GPU Devices: ", physical_devices)

logical_devices = tf.config.list_logical_devices('GPU')
print("Logical GPU Devices: ", logical_devices)


### Choice has been made to use `Coronal` view using `Processed` and proceed since it appears to be best performing.

In [ ]:
# Code to help choose files based on file choice and view choice
seedValue = 42 # Used across notebook where seed value has to be specified
n_splits = 5 # Used later for k-fold
coronalFile = 'processed_img_c.pickle'
labelsFile = 'all_labels_processed_c.pickle'
mriIDFile = 'all_mri_id_processed_c.pickle'  
with open("{}/{}".format(baseSharedPath,coronalFile), "rb") as f:
    img_16frames = pickle.load(f)
with open("{}/{}".format(baseSharedPath,labelsFile), "rb") as f:
    all_labels = pickle.load(f)
with open("{}/{}".format(baseSharedPath,mriIDFile), "rb") as f:
    all_mri_id = pickle.load(f)
        
# 'Stack' the images  since it currently is represented in form of a scalar's shape (x,). Needs to be (x,y,z)
img_16frames = np.stack(img_16frames, axis=0)
img_16frames = img_16frames.reshape(-1, img_16frames.shape[1], img_16frames.shape[2],1)

    #return(img_16frames,all_labels,all_mri_id)

# Code to standardize dataset
def standardizeImg(img_16frames):
    # Flatten the array along the last dimension
    img_16frames_flat = img_16frames.reshape(-1, img_16frames.shape[-1])
    # Standardize the flattened array
    scaler = StandardScaler()
    img_16frames_flat_scaled = scaler.fit_transform(img_16frames_flat)
    # Reshape the standardized array to its original shape and reassign to 'img_16frames'
    img_16frames = img_16frames_flat_scaled.reshape(img_16frames.shape)
    return img_16frames

# Standardize dataset
img_16frames = standardizeImg(img_16frames) # Standardize dataset
print(img_16frames.shape)

In [ ]:
# No longer needed since this is being done inside the KFold code below.
# X_train, X_test, y_train, y_test  = train_test_split(img_16frames, all_labels
# ,test_size=0.2, random_state=seedValue , stratify=all_labels)

In [ ]:
# Show sample image of one of the images based on the view that was picked

def plot_stitched_img(stitched_img):
    # takes arrays from get_mri_array function.
    # returns a sample of the image.
    plt.close();
    plt.figure(figsize=(50,30)) 
    plt.imshow(stitched_img, cmap=plt.cm.gray_r, interpolation="nearest") 
    plt.show()

#plot_stitched_img(img_16frames[0][:,:,0])

### Define re-usuable code to be able to generate model over each of the cross validation folds for each hyper-parameter choice

In [ ]:
# This is the base model over which iterations are actually being done
def generateModelFilter():
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu'
                    ,input_shape=(img_16frames.shape[1], img_16frames.shape[2], 1), name = "C_2d_1"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_2"))
    model.add(layers.Dropout(0.15))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_3"))
    model.add(layers.Dropout(0.15))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_4"))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "Dense_1"))
    model.add(layers.Dense(2))
    tf.random.set_seed(seedValue)
    model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001),
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])
    return model

# 1 - Re-usable code for generating model using filter kernel sizes in 1st layer
def generateModelKernelSize(kernelSize):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32, kernel_size= kernelSize
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu'
                    ,input_shape=(img_16frames.shape[1], img_16frames.shape[2], 1), name = "C_2d_1"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_2"))
    model.add(layers.Dropout(0.15))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_3"))
    model.add(layers.Dropout(0.15))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_4"))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "Dense_1"))
    model.add(layers.Dense(2))
    tf.random.set_seed(seedValue)
    model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001),
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])
    return model

# 2 - Re-usable code for generating model using number of filters in 1st layer
def generateModelFilter(numFilters):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=numFilters, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu'
                    ,input_shape=(img_16frames.shape[1], img_16frames.shape[2], 1), name = "C_2d_1"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_2"))
    model.add(layers.Dropout(0.15))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_3"))
    model.add(layers.Dropout(0.15))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_4"))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "Dense_1"))
    model.add(layers.Dense(2))
    tf.random.set_seed(seedValue)
    model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001),
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])
    return model

# 3 - Re-usable code for generating model using number of dense neurons in final layer
def generateModelDenseNeuron(denseNeuron):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu'
                    ,input_shape=(img_16frames.shape[1], img_16frames.shape[2], 1), name = "C_2d_1"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_2"))
    model.add(layers.Dropout(0.15))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_3"))
    model.add(layers.Dropout(0.15))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_4"))
    model.add(layers.Flatten())
    model.add(layers.Dense(denseNeuron, kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "Dense_1"))
    model.add(layers.Dense(2))
    tf.random.set_seed(seedValue)
    model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001),
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])
    return model

# 4 - Re-usable code for generating model using learning rate of model
def generateModelLearningRate(learnRate):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu'
                    ,input_shape=(img_16frames.shape[1], img_16frames.shape[2], 1), name = "C_2d_1"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_2"))
    model.add(layers.Dropout(0.15))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_3"))
    model.add(layers.Dropout(0.15))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_4"))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "Dense_1"))
    model.add(layers.Dense(2))
    tf.random.set_seed(seedValue)
    model.compile(tf.keras.optimizers.Adam(learning_rate=learnRate),
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])
    return model

# 5 - Re-usable code for generating model using l2 reg
def generateModelL2Reg(l2Reg):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(l2Reg), activation='relu'
                    ,input_shape=(img_16frames.shape[1], img_16frames.shape[2], 1), name = "C_2d_1"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(l2Reg), activation='relu', name = "C_2d_2"))
    model.add(layers.Dropout(0.15))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(l2Reg), activation='relu', name = "C_2d_3"))
    model.add(layers.Dropout(0.15))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(l2Reg), activation='relu', name = "C_2d_4"))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_regularizer = tf.keras.regularizers.L2(l2Reg), activation='relu', name = "Dense_1"))
    model.add(layers.Dense(2))
    tf.random.set_seed(seedValue)
    model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001),
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])
    return model

# 5 - Re-usable code for generating model using drop out choices in both the drop out layers.
def generateModelDropOut(dropOut):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu'
                    ,input_shape=(img_16frames.shape[1], img_16frames.shape[2], 1), name = "C_2d_1"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_2"))
    model.add(layers.Dropout(dropOut))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_3"))
    model.add(layers.Dropout(dropOut))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "C_2d_4"))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_regularizer = tf.keras.regularizers.L2(0.01), activation='relu', name = "Dense_1"))
    model.add(layers.Dense(2))
    tf.random.set_seed(seedValue)
    model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001),
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])
    return model

# Reusable code to generate f1_score, precision, recall and confusion matrix for a pair of X_test,y_test against a model
def generateValScores(model,X_test,y_test):
    y_pred = model.predict(X_test)
    y_pred = y_pred.round(1)
    y_pred_binary = [0 if x[0] > x[1] else 1 for x in y_pred]
    y_test_binary = list(y_test.reshape(1,-1)[0])
    f1Score = f1_score(y_test_binary, y_pred_binary, average='macro')
    precision = precision_score(y_test_binary, y_pred_binary)
    recall = recall_score(y_test_binary, y_pred_binary)
    accuracy = accuracy_score(y_test_binary, y_pred_binary)
    return (f1Score,precision,recall,accuracy)


### Sensitivity analysis 1 : Filter kernel sizes - 3,5,7,9,11

In [ ]:
# Main code cell for training over stratified k-fold data for KernelSizes
kernelSizes = [3,5,7,9,11]
kernelSenScores = [] # Empty list to store scores from each iteration
for kernelSize in kernelSizes:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seedValue) # Define the Stratified KFold object
    # Train for chosen view over k-folds using distributed strategy
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        # Iterate through each fold
        for i, (train_index, test_index) in enumerate(skf.split(img_16frames,all_labels)):
            print('Iteration: {} for Coronal for kernelSize={}'.format(i+1,kernelSize))
            # Generate model from scratch 
            model = generateModelKernelSize(kernelSize)
            checkpoint_filepath = "{}/{}".format(os.path.dirname(baseSharedPath),'tmp') # 'tmp' folder in shared space
            callbacks = [
                tf.keras.callbacks.ModelCheckpoint( # Saves best model on each epoch, which can be loaded and used
                    filepath=checkpoint_filepath, save_weights_only=True
                    ,monitor='val_accuracy',mode='max',save_best_only=True)
            ]
            # Generate new datasets for X_train, X_test, y_train, y_test on each run
            X_train, X_test =  img_16frames[train_index],img_16frames[test_index]
            y_train, y_test = all_labels[train_index],all_labels[test_index]
            model.fit(X_train,  y_train, epochs=20, 
                            validation_data=(X_test, y_test),callbacks=callbacks,verbose=2)
            history = model.load_weights(checkpoint_filepath) # Load best model that is available from ModelCheckpoint callback data
            # Store the evaluation metrics for this fold
            kernelSenScores.append([kernelSize,i+1,*generateValScores(model,X_test,y_test)])

In [ ]:
# Pickle results with appropriate name so that it can be loaded and used if needed
runDateTime = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
with open("{}/kernelSenScores_{}fold_seed{}_{}.pickle".format(baseSharedPath+'/sen_analysis',n_splits,seedValue,runDateTime), "wb") as f:
    pickle.dump(kernelSenScores, f)

### Sensitivity analysis 2 : Number of filters - 8,16,32,64,128,256

In [ ]:
# Main code cell for training over stratified k-fold data for number of filters
numFiltersChoices = [8,16,32,64,128,256]
numFiltersSenScores = [] # Empty list to store scores from each iteration
for numFilters in numFiltersChoices:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seedValue) # Define the Stratified KFold object
    # Train for chosen view over k-folds using distributed strategy
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        # Iterate through each fold
        for i, (train_index, test_index) in enumerate(skf.split(img_16frames,all_labels)):
            print('Iteration: {} for Coronal for numFilters={}'.format(i+1,numFilters))
            # Generate model from scratch 
            model = generateModelFilter(numFilters)
            checkpoint_filepath = "{}/{}".format(os.path.dirname(baseSharedPath),'tmp') # 'tmp' folder in shared space
            callbacks = [
                tf.keras.callbacks.ModelCheckpoint( # Saves best model on each epoch, which can be loaded and used
                    filepath=checkpoint_filepath, save_weights_only=True
                    ,monitor='val_accuracy',mode='max',save_best_only=True)
            ]
            # Generate new datasets for X_train, X_test, y_train, y_test on each run
            X_train, X_test =  img_16frames[train_index],img_16frames[test_index]
            y_train, y_test = all_labels[train_index],all_labels[test_index]
            model.fit(X_train,  y_train, epochs=20, 
                            validation_data=(X_test, y_test),callbacks=callbacks,verbose=2)
            history = model.load_weights(checkpoint_filepath) # Load best model that is available from ModelCheckpoint callback data
            # Store the evaluation metrics for this fold
            numFiltersSenScores.append([numFilters,i+1,*generateValScores(model,X_test,y_test)])

In [ ]:
# Pickle results with appropriate name so that it can be loaded and used if needed
runDateTime = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
with open("{}/numFiltersSenScores_{}fold_seed{}_{}.pickle".format(baseSharedPath+'/sen_analysis',n_splits,seedValue,runDateTime), "wb") as f:
    pickle.dump(numFiltersSenScores, f)

### Sensitivity analysis 3 : Neurons in fully connected layer - 16,32,64,128,256

In [ ]:
# Main code cell for training over stratified k-fold data for dense layer neurons
denseNeuronChoices = [8,16,32,64,128,256]
denseNeuronSenScores = [] # Empty list to store scores from each iteration
for denseNeuron in denseNeuronChoices:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seedValue) # Define the Stratified KFold object
    # Train for chosen view over k-folds using distributed strategy
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        # Iterate through each fold
        for i, (train_index, test_index) in enumerate(skf.split(img_16frames,all_labels)):
            print('Iteration: {} for Coronal for denseNeuron={}'.format(i+1,denseNeuron))
            # Generate model from scratch 
            model = generateModelDenseNeuron(denseNeuron)
            checkpoint_filepath = "{}/{}".format(os.path.dirname(baseSharedPath),'tmp') # 'tmp' folder in shared space
            callbacks = [
                tf.keras.callbacks.ModelCheckpoint( # Saves best model on each epoch, which can be loaded and used
                    filepath=checkpoint_filepath, save_weights_only=True
                    ,monitor='val_accuracy',mode='max',save_best_only=True)
            ]
            # Generate new datasets for X_train, X_test, y_train, y_test on each run
            X_train, X_test =  img_16frames[train_index],img_16frames[test_index]
            y_train, y_test = all_labels[train_index],all_labels[test_index]
            model.fit(X_train,  y_train, epochs=20, 
                            validation_data=(X_test, y_test),callbacks=callbacks,verbose=2)
            history = model.load_weights(checkpoint_filepath) # Load best model that is available from ModelCheckpoint callback data
            # Store the evaluation metrics for this fold
            denseNeuronSenScores.append([denseNeuron,i+1,*generateValScores(model,X_test,y_test)])

In [ ]:
# Pickle results with appropriate name so that it can be loaded and used if needed
runDateTime = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
with open("{}/denseNeuronSenScores_{}fold_seed{}_{}.pickle".format(baseSharedPath+'/sen_analysis',n_splits,seedValue,runDateTime), "wb") as f:
    pickle.dump(denseNeuronSenScores, f)

### Sensitivity analysis 4 : Learning rate of model - 0.0001,0.001,0.01,0.1,1

In [ ]:
# Main code cell for training over stratified k-fold data for learning rate
learnRateChoices = [0.0001,0.001,0.01,0.1,1]
learnRateSenScores = [] # Empty list to store scores from each iteration
for learnRate in learnRateChoices:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seedValue) # Define the Stratified KFold object
    # Train for chosen view over k-folds using distributed strategy
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        # Iterate through each fold
        for i, (train_index, test_index) in enumerate(skf.split(img_16frames,all_labels)):
            print('Iteration: {} for Coronal for learnRate={}'.format(i+1,learnRate))
            # Generate model from scratch 
            model = generateModelLearningRate(learnRate)
            checkpoint_filepath = "{}/{}".format(os.path.dirname(baseSharedPath),'tmp') # 'tmp' folder in shared space
            callbacks = [
                tf.keras.callbacks.ModelCheckpoint( # Saves best model on each epoch, which can be loaded and used
                    filepath=checkpoint_filepath, save_weights_only=True
                    ,monitor='val_accuracy',mode='max',save_best_only=True)
            ]
            # Generate new datasets for X_train, X_test, y_train, y_test on each run
            X_train, X_test =  img_16frames[train_index],img_16frames[test_index]
            y_train, y_test = all_labels[train_index],all_labels[test_index]
            model.fit(X_train,  y_train, epochs=20, 
                            validation_data=(X_test, y_test),callbacks=callbacks,verbose=2)
            history = model.load_weights(checkpoint_filepath) # Load best model that is available from ModelCheckpoint callback data
            # Store the evaluation metrics for this fold
            learnRateSenScores.append([learnRate,i+1,*generateValScores(model,X_test,y_test)])

In [ ]:
# Pickle results with appropriate name so that it can be loaded and used if needed
runDateTime = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
with open("{}/learnRateSenScores_{}fold_seed{}_{}.pickle".format(baseSharedPath+'/sen_analysis',n_splits,seedValue,runDateTime), "wb") as f:
    pickle.dump(learnRateSenScores, f)

### Sensitivity Analysis 5 - L2 regularizer choices that are applied several layers

In [ ]:
# Main code cell for training over stratified k-fold data for l2 reg
l2RegChoices = [0.001,0.005,0.01,0.05,0.1,0.2,0.5]
l2RegSenScores = [] # Empty list to store scores from each iteration
for l2Reg in l2RegChoices:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seedValue) # Define the Stratified KFold object
    # Train for chosen view over k-folds using distributed strategy
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        # Iterate through each fold
        for i, (train_index, test_index) in enumerate(skf.split(img_16frames,all_labels)):
            print('Iteration: {} for Coronal for l2Reg={}'.format(i+1,l2Reg))
            # Generate model from scratch 
            model = generateModelL2Reg(l2Reg)
            checkpoint_filepath = "{}/{}".format(os.path.dirname(baseSharedPath),'tmp') # 'tmp' folder in shared space
            callbacks = [
                tf.keras.callbacks.ModelCheckpoint( # Saves best model on each epoch, which can be loaded and used
                    filepath=checkpoint_filepath, save_weights_only=True
                    ,monitor='val_accuracy',mode='max',save_best_only=True)
            ]
            # Generate new datasets for X_train, X_test, y_train, y_test on each run
            X_train, X_test =  img_16frames[train_index],img_16frames[test_index]
            y_train, y_test = all_labels[train_index],all_labels[test_index]
            model.fit(X_train,  y_train, epochs=20, 
                            validation_data=(X_test, y_test),callbacks=callbacks,verbose=2)
            history = model.load_weights(checkpoint_filepath) # Load best model that is available from ModelCheckpoint callback data
            # Store the evaluation metrics for this fold
            l2RegSenScores.append([l2Reg,i+1,*generateValScores(model,X_test,y_test)])

In [ ]:
# Pickle results with appropriate name so that it can be loaded and used if needed
runDateTime = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
with open("{}/l2RegSenScores_{}fold_seed{}_{}.pickle".format(baseSharedPath+'/sen_analysis',n_splits,seedValue,runDateTime), "wb") as f:
    pickle.dump(l2RegSenScores, f)

### Sensitivity Analysis 6 - Drop out fractions applied on the 2 drop out layers

In [ ]:
# Main code cell for training over stratified k-fold data for drop out
dropOutChoices = [0.05,0.1,0.2,0.3,0.4,0.5]
dropOutSenScores = [] # Empty list to store scores from each iteration
for dropOut in dropOutChoices:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seedValue) # Define the Stratified KFold object
    # Train for chosen view over k-folds using distributed strategy
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        # Iterate through each fold
        for i, (train_index, test_index) in enumerate(skf.split(img_16frames,all_labels)):
            print('Iteration: {} for Coronal for dropOut={}'.format(i+1,dropOut))
            # Generate model from scratch 
            model = generateModelDropOut(dropOut)
            checkpoint_filepath = "{}/{}".format(os.path.dirname(baseSharedPath),'tmp') # 'tmp' folder in shared space
            callbacks = [
                tf.keras.callbacks.ModelCheckpoint( # Saves best model on each epoch, which can be loaded and used
                    filepath=checkpoint_filepath, save_weights_only=True
                    ,monitor='val_accuracy',mode='max',save_best_only=True)
            ]
            # Generate new datasets for X_train, X_test, y_train, y_test on each run
            X_train, X_test =  img_16frames[train_index],img_16frames[test_index]
            y_train, y_test = all_labels[train_index],all_labels[test_index]
            model.fit(X_train,  y_train, epochs=20, 
                            validation_data=(X_test, y_test),callbacks=callbacks,verbose=2)
            history = model.load_weights(checkpoint_filepath) # Load best model that is available from ModelCheckpoint callback data
            # Store the evaluation metrics for this fold
            dropOutSenScores.append([dropOut,i+1,*generateValScores(model,X_test,y_test)])



In [ ]:
# Pickle results with appropriate name so that it can be loaded and used if needed
runDateTime = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
with open("{}/dropOutSenScores_{}fold_seed{}_{}.pickle".format(baseSharedPath+'/sen_analysis',n_splits,seedValue,runDateTime), "wb") as f:
    pickle.dump(dropOutSenScores, f)